## Netflix DB

In [1]:
# Import libraries

import pandas as pd
from sqlalchemy import create_engine
import os
from config import omdb_key, Posgres_Pswrd
import numpy as np
import requests
import json

In [2]:
# Loading CSV file into Pandas

csv_file = os.path.join("netflix_titles.csv")
netflix_df = pd.read_csv(csv_file)
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
# Exploring Netflix_titles file

netflix_df.shape

(6234, 12)

In [4]:
sorted(netflix_df)

['cast',
 'country',
 'date_added',
 'description',
 'director',
 'duration',
 'listed_in',
 'rating',
 'release_year',
 'show_id',
 'title',
 'type']

In [5]:
# Identifing missing fields

netflix_df['director'].isnull().sum()

1969

In [6]:
netflix_df['cast'].isnull().sum()

570

In [7]:
netflix_df['country'].isnull().sum()

476

In [8]:
netflix_df['date_added'].isnull().sum()

11

In [9]:
netflix_df['rating'].isnull().sum()

10

In [10]:
# Renaming columns

netflix_df.columns = ['show_id', 'show_type', 'show_title', 'show_director', 'show_cast', 'show_country', 'date_added', 'release_year', 'show_rating', 'show_duration', 'listed_in', 'show_description']
netflix_df.head()

,show_id,show_type,show_title,show_director,show_cast,show_country,date_added,release_year,show_rating,show_duration,listed_in,show_description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [11]:
# Assigning data types

# netflix_df.dtypes
netflix_df.astype({'date_added': 'datetime64[ns]'}).dtypes

show_id                      int64
show_type                   object
show_title                  object
show_director               object
show_cast                   object
show_country                object
date_added          datetime64[ns]
release_year                 int64
show_rating                 object
show_duration               object
listed_in                   object
show_description            object
dtype: object

In [12]:
# Replacing Null values with blank field

netflix_df.fillna('')

,show_id,show_type,show_title,show_director,show_cast,show_country,date_added,release_year,show_rating,show_duration,listed_in,show_description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
...,...,...,...,...,...,...,...,...,...,...,...,...
6229,80000063,TV Show,Red vs. Blue,,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil..."
6230,70286564,TV Show,Maron,,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews..."
6231,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,,,,,2016,,60 min,Movies,Nursery rhymes and original music for children...
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."


In [13]:
netflix_drop_df = netflix_df.drop_duplicates(subset="show_title", keep="first")

In [14]:
netflix_drop_df.shape

(6172, 12)

In [15]:
not_found = []
om_actors= []
om_awards= []
om_country= []
om_director= []
om_genre= []
om_language= []
om_metascore= []
om_plot= []
om_poster= []
om_rated= []
om_released= []
om_response= []
om_runtime= []
om_title= []
om_type= []
om_writer = []
om_year= []
om_imdbID= []
om_imdbVotes = []
om_rating_imdb= []
om_rating_rotten= []
om_rating_metacritic = []


print("Start Retrieval")    
for movie in netflix_drop_df['show_title']:
#     print(movie)
    url = f'http://www.omdbapi.com/?t={movie}&apikey={omdb_key}' 
    response = requests.get(url)
    data = response.json()
    
    if data['Response']=='True':
#         print(f'found {data["Title"]}')
        om_actors.append(data['Actors'])
        om_awards.append(data['Awards'])
        om_country.append(data['Country'])
        om_director.append(data['Director'])
        om_genre.append(data['Genre'])
        om_language.append(data['Language'])
        om_metascore.append(data['Metascore'])
        om_plot.append(data['Plot'])
        om_poster.append(data['Poster'])
        om_rated.append(data['Rated'])
        om_released.append(data['Released'])
        om_response.append(data['Response'])
        om_runtime.append(data['Runtime'])
        om_title.append(data['Title'])
        om_type.append(data['Type'])
        om_writer.append(data['Writer'])
        om_year.append(data['Year'])
        om_imdbID.append(data['imdbID'])
        om_imdbVotes.append(data['imdbVotes'])

        rating1 = ''
        rating2 = ''
        rating3 = ''

        ratings = data['Ratings']
        for rating in ratings:
            if rating['Source'] == 'Internet Movie Database':
                rating1 = rating['Value']
            elif rating['Source'] == 'Rotten Tomatoes':
                rating2 = rating['Value']
            else:
                rating3 = rating['Value']

        om_rating_imdb.append(rating1)
        om_rating_rotten.append(rating2)
        om_rating_metacritic.append(rating3)


    else:
        not_found.append(movie)
        
print('Finished Retrieval')

Start Retrieval
Finished Retrieval


In [16]:
print(f'Actors: {len(om_actors)}')
print(f'Awards: {len(om_awards)}')
print(f'Country: {len(om_country)}')
print(f'Director: {len(om_director)}')
print(f'Genre: {len(om_genre)}')
print(f'Language: {len(om_language)}')
print(f'Metascore: {len(om_metascore)}')
print(f'Plot: {len(om_plot)}')
print(f'Poster: {len(om_poster)}')
print(f'Rated: {len(om_rated)}')
print(f'Released: {len(om_released)}')
print(f'Response: {len(om_response)}')
print(f'Runtime: {len(om_runtime)}')
print(f'Title: {len(om_title)}')
print(f'Type: {len(om_type)}')
print(f'Writer: {len(om_writer)}')
print(f'Year: {len(om_year)}')
print(f'Imdb ID: {len(om_imdbID)}')
print(f'Imdb Votes: {len(om_imdbVotes)}')
print(f'Rating Imdb: {len(om_rating_imdb)}')
print(f'Rating Rotten: {len(om_rating_rotten)}')
print(f'Rating Metacritc: {len(om_rating_metacritic)}')

print(f'Number of Movies not in OMDB: {len(not_found)}')

Actors: 869
Awards: 869
Country: 869
Director: 869
Genre: 869
Language: 869
Metascore: 869
Plot: 869
Poster: 869
Rated: 869
Released: 869
Response: 869
Runtime: 869
Title: 869
Type: 869
Writer: 869
Year: 869
Imdb ID: 869
Imdb Votes: 869
Rating Imdb: 869
Rating Rotten: 869
Rating Metacritc: 869
Number of Movies not in OMDB: 5303


In [17]:
open_movie_dict = {
    'actor': om_actors,
    'awards': om_awards,
    'country': om_country,
    'director': om_director,
    'genre': om_genre,
    'language': om_language,
    'metascore': om_metascore,
    'plot': om_plot,
    'poster': om_poster,
    'rated': om_rated,
    'released': om_released,
    'response': om_response,
    'runtime': om_runtime,
    'title': om_title,
    'type': om_type,
    'writer': om_writer,
    'year': om_year,
    'imdb_id': om_imdbID,
    'imdb_votes': om_imdbVotes,
    'rating_imdb': om_rating_imdb,
    'rating_rotten': om_rating_rotten,
    'rating_metacritic': om_rating_metacritic
    }
open_movie_df = pd.DataFrame(open_movie_dict)
open_movie_df.shape

# out_file = os.path.join('open_movie.csv')
# open_movie_df.to_csv(out_file, index=False)

(869, 22)

In [18]:
duplicate = open_movie_df[open_movie_df.duplicated('title')] 
  
print("Duplicate Rows based on Show Title :")
duplicate

Duplicate Rows based on Show Title :


,actor,awards,country,director,genre,language,metascore,plot,poster,rated,...,runtime,title,type,writer,year,imdb_id,imdb_votes,rating_imdb,rating_rotten,rating_metacritic


In [19]:
open_movie_drop_df = open_movie_df.drop_duplicates(subset="title", keep="first")
open_movie_drop_df.shape

(869, 22)

In [20]:
movies_combined = pd.merge(netflix_drop_df, open_movie_drop_df, left_on=['show_title'], right_on=['title'], how='left')
movies_combined.head()

,show_id,show_type,show_title,show_director,show_cast,show_country,date_added,release_year,show_rating,show_duration,...,runtime,title,type,writer,year,imdb_id,imdb_votes,rating_imdb,rating_rotten,rating_metacritic
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,...,90 min,Norm of the North: King Sized Adventure,movie,Dean Stefan,2019,tt9428190,290,3.2/10,36%,
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,...,95 min,Jandino: Whatever it Takes,movie,N/A,2016,tt6999080,15,5.4/10,,
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,...,30 min,Transformers Prime,series,N/A,2010–2013,tt1659175,"5,258",7.9/10,,
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,...,22 min,Transformers: Robots in Disguise,series,N/A,2014–2020,tt3604232,799,6.0/10,,
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
movies_limited = movies_combined[['show_id', 'show_type', 'show_title', 'show_director', 'show_cast',
                               'show_country', 'date_added', 'release_year', 'show_rating', 'show_duration',
                               'listed_in', 'show_description', 'awards', 'genre', 'language', 'writer',
                               'imdb_id', 'imdb_votes', 'rating_imdb', 'rating_rotten', 'rating_metacritic']]
movies_limited.tail()

,show_id,show_type,show_title,show_director,show_cast,show_country,date_added,release_year,show_rating,show_duration,...,show_description,awards,genre,language,writer,imdb_id,imdb_votes,rating_imdb,rating_rotten,rating_metacritic
6167,80159925,TV Show,Kikoriki,NaN,Igor Dmitriev,NaN,NaN,2010,TV-Y,2 Seasons,...,A wacky rabbit and his gang of animal pals hav...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6168,80000063,TV Show,Red vs. Blue,NaN,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,NaN,2015,NR,13 Seasons,...,"This parody of first-person shooter games, mil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6169,70286564,TV Show,Maron,NaN,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,NaN,2016,TV-MA,4 Seasons,...,"Marc Maron stars as Marc Maron, who interviews...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6170,70281022,TV Show,A Young Doctor's Notebook and Other Stories,NaN,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaN,2013,TV-MA,2 Seasons,...,"Set during the Russian Revolution, this comic ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6171,70153404,TV Show,Friends,NaN,"Jennifer Aniston, Courteney Cox, Lisa Kudrow, ...",United States,NaN,2003,TV-14,10 Seasons,...,This hit sitcom follows the merry misadventure...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
duplicate = movies_limited[movies_limited.duplicated('show_id')] 
  
print("Duplicate Rows based on Show IDs :")

Duplicate Rows based on Show IDs :


In [23]:
# Creating Posgres Engine

pg_user = 'postgres'
pg_password = Posgres_Pswrd
db_name = 'movies_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [24]:
engine.table_names()

['netflix_omdb']

In [25]:
movies_limited.to_sql(name='netflix_omdb', con=engine, if_exists='append', index=False)

In [26]:
pd.read_sql_query('select * from netflix_omdb', con=engine).head()

,show_id,show_type,show_title,show_director,show_cast,show_country,date_added,release_year,show_rating,show_duration,...,show_description,awards,genre,language,writer,imdb_id,imdb_votes,rating_imdb,rating_rotten,rating_metacritic
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09,2019,TV-PG,90 min,...,Before planning an awesome wedding for his gra...,N/A,"Animation, Adventure, Comedy, Family",English,Dean Stefan,tt9428190,290,3.2/10,36%,
1,80117401,Movie,Jandino: Whatever it Takes,None,Jandino Asporaat,United Kingdom,2016-09-09,2016,TV-MA,94 min,...,Jandino Asporaat riffs on the challenges of ra...,N/A,Comedy,"English, Dutch",N/A,tt6999080,15,5.4/10,,
2,70234439,TV Show,Transformers Prime,None,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09-08,2013,TV-Y7-FV,1 Season,...,"With the help of three human allies, the Autob...",14 wins & 26 nominations.,"Animation, Action, Adventure, Comedy, Drama, F...",English,N/A,tt1659175,"5,258",7.9/10,,
3,80058654,TV Show,Transformers: Robots in Disguise,None,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09-08,2016,TV-Y7,1 Season,...,When a prison ship crash unleashes hundreds of...,2 wins & 11 nominations.,"Animation, Action, Adventure, Comedy, Sci-Fi",English,N/A,tt3604232,799,6.0/10,,
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09-08,2017,TV-14,99 min,...,When nerdy high schooler Dani finally attracts...,None,None,None,None,None,None,None,None,None
